# Shorの符号で論理ビット操作

このノートブックでは、符号化された状態で、各種ゲート操作に対応する操作を実行する論理ビット操作を行う。

ここでは、先ほどのノートブックで利用したshorの符号を用いる。

In [ ]:
%pip install qiskit qiskit-aer
%pip install pylatexenc

In [ ]:
import random
import numpy as np

from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit_aer import AerSimulator
from qiskit.visualization import plot_histogram
from qiskit_aer.noise import pauli_error
from qiskit_aer.noise.errors.base_quantum_error import QuantumChannelInstruction
from qiskit.circuit.library import U3Gate

# ベルンシュタイン・ヴァジラニのアルゴリズム

ここでは、簡単な回路の例として、ベルンシュタイン・ヴァジラニのアルゴリズムを動かしてみる。

回路としては以下のような形になる。

In [ ]:
backend_sim = AerSimulator()
n_shots = 1000

In [ ]:
n = 2
bv = QuantumCircuit(n+1, n)

bv.x(n)
bv.h(range(n+1))

bv.barrier()
# オラクルを回路に追加する
bv.cx(0, n)
bv.barrier()

bv.h(range(n+1))

# 測定する
bv.measure(range(n), range(n))
bv.draw("mpl")

In [ ]:
result_ideal = backend_sim.run(bv, shots=n_shots).result()
plot_histogram(result_ideal.get_counts(0))

これをshorの符号を用いた論理ビット操作で実現する。

In [ ]:
def shor_encode() -> QuantumCircuit:
    # 回路の記述
    n_qubits = 9
    circ = QuantumCircuit(n_qubits)

    # 符号化
    circ.cx(0, 3)
    circ.cx(0, 6)

    circ.h(0)
    circ.h(3)
    circ.h(6)

    circ.cx(0, 1)
    circ.cx(0, 2)

    circ.cx(3, 4)
    circ.cx(3, 5)

    circ.cx(6, 7)
    circ.cx(6, 8)

    return circ

In [ ]:
def shor_decode() -> QuantumCircuit:
    # 回路の記述
    n_qubits = 9
    circ = QuantumCircuit(n_qubits)

    # 復号
    circ.cx(6, 8)
    circ.cx(6, 7)

    circ.cx(3, 5)
    circ.cx(3, 4)

    circ.cx(0, 2)
    circ.cx(0, 1)

    circ.ccx(2, 1, 0)
    circ.ccx(5, 4, 3)
    circ.ccx(8, 7, 6)

    circ.h(0)
    circ.h(3)
    circ.h(6)

    circ.cx(0, 6)
    circ.cx(0, 3)
    circ.ccx(6, 3, 0)

    return circ

ここで、各種ゲート操作に対応する論理ビット操作の実装を準備する。

In [ ]:
def shor_cnot(circ: QuantumCircuit, qr1: QuantumRegister, qr2: QuantumRegister) -> QuantumCircuit:
    logical_cnot = QuantumCircuit(18)

    for n in range(9):
        logical_cnot.cx(n, n+9)

    circ.append(logical_cnot, qr1 + qr2)
    return circ

def shor_x() -> QuantumCircuit:
    circ = QuantumCircuit(9)
    circ.z(2)
    circ.z(5)
    circ.z(8)
    return circ

In [ ]:
def shor_hadamard() -> QuantumCircuit:
    circ = QuantumCircuit(9)
    circ.h(0)
    circ.h(1)
    circ.append(U3Gate(0.5 * np.pi, 0.5 * np.pi, 0.5 * np.pi), [2])
    circ.cx(0,1)
    circ.cx(1,2)
    circ.cx(2,5)
    circ.cx(5,8)
    circ.rz(0.5 * np.pi, 8)
    circ.cx(5,8)
    circ.cx(2,5)
    circ.cx(1,2)
    circ.cx(0,1)
    circ.h(0)
    circ.h(1)
    circ.append(U3Gate(0.5 * np.pi, 0.5 * np.pi, 0.5 * np.pi), [2])
    circ.x(0)
    circ.x(1)
    circ.x(2)

    return circ

それぞれの操作を可視化してみると、以下のようになる

## 論理Xゲート

In [ ]:
shor_x().draw("mpl")

## 論理アダマールゲート

In [ ]:
shor_hadamard().draw("mpl")

## 論理CNOTゲート

In [ ]:
qr = QuantumRegister(18, 'q')
circ = QuantumCircuit(qr)
shor_cnot(circ, qr[9:], qr[0:9]).decompose().draw("mpl")

これらの要素を組み合わせて、元のアルゴリズムに相当する量子回路を構成すると以下のようになる

In [ ]:
def make_logical_bv(x_error_channel: list[int] = [], z_error_channel: list[int] = []):

    shor_encode_instruction = shor_encode().to_instruction()
    shor_decode_instruction = shor_decode().to_instruction()
    shor_x_instruction = shor_x().to_instruction()
    shor_hadamard_instruction = shor_hadamard().to_instruction()
    shor_encode_instruction.name = "encode"
    shor_decode_instruction.name = "decode"
    shor_x_instruction.name = "logical_x"
    shor_hadamard_instruction.name = "logical_hadamard"

    qr = QuantumRegister(27, 'q')
    cr = ClassicalRegister(2, 'c')
    circ = QuantumCircuit(qr, cr)

    # エンコード
    circ.append(shor_encode_instruction, qr[0:9])
    circ.append(shor_encode_instruction, qr[9:18])
    circ.append(shor_encode_instruction, qr[18:])
    circ.barrier()

    # エラーチャネル
    for i in x_error_channel:
        circ.x(i)

    for i in z_error_channel:
        circ.z(i)

    circ.barrier()

    # 論理アダマールと反転
    circ.append(shor_hadamard_instruction, qr[0:9])
    circ.append(shor_hadamard_instruction, qr[9:18])
    circ.append(shor_x_instruction, qr[18:])
    circ.append(shor_hadamard_instruction, qr[18:])
    circ.barrier()

    # 論理CNOT
    shor_cnot(circ, qr[18:], qr[0:9])
    circ.barrier()

    # 論理アダマール
    circ.append(shor_hadamard_instruction, qr[0:9])
    circ.append(shor_hadamard_instruction, qr[9:18])
    circ.barrier()

    # デコード
    circ.append(shor_decode_instruction, qr[0:9])
    circ.append(shor_decode_instruction, qr[9:18])
    circ.append(shor_decode_instruction, qr[18:])
    circ.barrier()

    # 観測
    circ.measure(qr[0], cr[0])
    circ.measure(qr[9], cr[1])

    return circ

In [ ]:
circ = make_logical_bv()

抽象化された回路を表示すると、以下のようになる。

量子ビット数こそ多いものの、先ほどの回路と同じ操作になっている。

In [ ]:
circ.draw("mpl")

細かいゲート操作に書き下してみると、以下のようになる

In [ ]:
circ.decompose().draw("mpl")

作成した回路を動かして、結果を確認してみる。
まずは、エラーをかけない状態で動かす。

In [ ]:
result_ideal = backend_sim.run(circ.decompose(), shots=n_shots).result()
plot_histogram(result_ideal.get_counts(0))

ここに、1量子ビットまでのエラーを加えてもうまく訂正できるのが期待値であるが、実際にはうまくいかないケースが出てくる。

In [ ]:
# うまくいく場合
circ_noise = make_logical_bv(x_error_channel=[0])

result_noise = backend_sim.run(circ_noise.decompose(), shots=n_shots).result()
plot_histogram(result_noise.get_counts(0))

In [ ]:
# うまくいかない場合
circ_noise = make_logical_bv(z_error_channel=[18])

result_noise = backend_sim.run(circ_noise.decompose(), shots=n_shots).result()
plot_histogram(result_noise.get_counts(0))

正しく動いている時は$|10>$が得られているのに対し、$|00>$が出力されている。